# Figure 4
This file produces the results for "Figure 4. Commute Mode in Fruitvale".  

In [1]:
#Import packages
import pandas as pd
import matplotlib as mpl
import numpy as np
import math
import os
pd.options.mode.chained_assignment = None #gets rid of warning for chained alterations

In [2]:
#set directories
os.chdir("/Users/briangoggin/Dropbox/CP 201A/Fruitvale")
cwd = os.getcwd()
root = cwd+"/Raw Data/" #root is directory to raw files
root

'/Users/briangoggin/Dropbox/CP 201A/Fruitvale/Raw Data/'

In [3]:
#set input data source
commute_mode = root+"ACS_14_5YR_B08101_with_ann.csv"


In [4]:
#identify Fruitvale Census Tracts
Fruitvale = [6001406100, 6001406201, 6001406202, 6001406300, 6001406500, 6001407101, 6001407102, 6001407200, 6001407300, 6001407400, 6001407500, 6001407600]


In [5]:
#Define function for import and standard ACS cleaning operations
def import_census(file):
    df = pd.read_csv(file, header = 1)
    df['Fruitvale'] = df['Id2'].isin(Fruitvale)
    #the following 5 lines create duplicate observations of Fruitvale to include them in county averages
    df2 = df[df['Fruitvale'] == True]
    df2.drop('Fruitvale', axis = 1)
    df2.is_copy = False
    df2['Fruitvale'] = False
    df = df.append(df2)
    return df

#Define function for proportions MOE
def prop_MOE(numerator, denominator, num_moe, den_moe):
    return ((num_moe**2 - (((numerator/denominator)**2)*(den_moe**2))))**(1/2)/denominator

In [6]:
#import data
commute_mode = import_census(commute_mode)

In [7]:
#Convert all commute mode variables to integers
commute_mode['total_pop'] = commute_mode["Estimate; Total:"].astype(int)
commute_mode['total_pop_moe'] = commute_mode["Margin of Error; Total:"].astype(int)
commute_mode['car_alone'] = commute_mode["Estimate; Total: - Car, truck, or van - drove alone:"].astype(int)
commute_mode['car_alone_moe'] = commute_mode["Margin of Error; Total: - Car, truck, or van - drove alone:"].astype(int)
commute_mode['publict'] = commute_mode["Estimate; Total: - Public transportation (excluding taxicab):"].astype(int)
commute_mode['publict_moe'] = commute_mode["Margin of Error; Total: - Public transportation (excluding taxicab):"].astype(int)
commute_mode['carpool'] = commute_mode["Estimate; Total: - Car, truck, or van - carpooled:"].astype(int)
commute_mode['carpool_moe'] = commute_mode["Margin of Error; Total: - Car, truck, or van - carpooled:"].astype(int)
commute_mode['walk'] = commute_mode["Estimate; Total: - Walked:"].astype(int)
commute_mode['walk_moe'] = commute_mode["Margin of Error; Total: - Walked:"].astype(int)
commute_mode['taxi_bike'] = commute_mode["Estimate; Total: - Taxicab, motorcycle, bicycle, or other means:"].astype(int)
commute_mode['taxi_bike_moe'] = commute_mode["Margin of Error; Total: - Taxicab, motorcycle, bicycle, or other means:"].astype(int)
commute_mode['home'] = commute_mode["Estimate; Total: - Worked at home:"].astype(int)
commute_mode['home_moe'] = commute_mode["Margin of Error; Total: - Worked at home:"].astype(int)


In [8]:
#list moes to transform
list = ['total_pop_moe', 'publict_moe', 'car_alone_moe', 
               'carpool_moe', 'walk_moe', 'taxi_bike_moe', 
              'home_moe']

#convert MOEs to 95% confidence level
def convert(column):
    return column*(1.96/1.645)

for item in list:
    commute_mode[item] = convert(commute_mode[item])

In [9]:
#Create Commute standard MOEs

#square MOEs for aggregate MOEs
commute_mode['total_pop_moe2'] = (commute_mode['total_pop_moe'])**2
commute_mode['publict_moe2'] = (commute_mode['publict_moe'])**2
commute_mode['car_alone_moe2'] = (commute_mode['car_alone_moe'])**2
commute_mode['carpool_moe2'] = (commute_mode['carpool_moe'])**2
commute_mode['walk_moe2'] = (commute_mode['walk_moe'])**2
commute_mode['taxi_bike_moe2'] = (commute_mode['taxi_bike_moe'])**2
commute_mode['home_moe2'] = (commute_mode['home_moe'])**2

#create 'other' category, its moe, and its moe squared
commute_mode['other'] = commute_mode['walk'] + commute_mode['taxi_bike'] + commute_mode['home']
commute_mode['other_moe'] = (commute_mode['walk_moe2'] + commute_mode['taxi_bike_moe2'] + commute_mode['home_moe2'])**(1/2) 
commute_mode['other_moe2'] = (commute_mode['other_moe'])**2


#list data to include in sum
commutes = ['total_pop', 'publict', 'car_alone', 'carpool', 'other',
            'total_pop_moe2', 'publict_moe2', 'car_alone_moe2', 'carpool_moe2', 'other_moe2']

#Sum data 
excommutes = commute_mode.groupby('Fruitvale')[commutes].sum()

excommutes['total_pop_moe'] = np.sqrt(excommutes['total_pop_moe2'])
excommutes['pct_transit'] = (excommutes['publict']/excommutes['total_pop'])
excommutes['publict_moe'] = np.sqrt(excommutes['publict_moe2'])
excommutes['pct_car_alone'] = (excommutes['car_alone']/excommutes['total_pop'])
excommutes['car_alone_moe'] = np.sqrt(excommutes['car_alone_moe2'])
excommutes['pct_carpool'] = (excommutes['carpool']/excommutes['total_pop'])
excommutes['carpool_moe'] = np.sqrt(excommutes['carpool_moe2'])
excommutes['pct_other'] = (excommutes['other']/excommutes['total_pop'])
excommutes['other_moe'] = np.sqrt(excommutes['other_moe2'])

excommutes

In [10]:
#generate proportion MOEs
excommutes['pct_transit_moe'] = prop_MOE(excommutes['publict'], excommutes['total_pop'], excommutes['publict_moe'], excommutes['total_pop_moe'])
excommutes['pct_car_alone_moe'] = prop_MOE(excommutes['car_alone'], excommutes['total_pop'], excommutes['car_alone_moe'], excommutes['total_pop_moe'])
excommutes['pct_carpool_moe'] = prop_MOE(excommutes['carpool'], excommutes['total_pop'], excommutes['carpool_moe'], excommutes['total_pop_moe'])
excommutes['pct_other_moe'] = prop_MOE(excommutes['other'], excommutes['total_pop'], excommutes['other_moe'], excommutes['total_pop_moe'])


In [11]:
#set variables to keep 
keep_vars = ['pct_car_alone', 'pct_transit', 
             'pct_carpool', 'pct_other', 
             'pct_car_alone_moe', 'pct_transit_moe', 
             'pct_carpool_moe', 'pct_other_moe']

excommutes = excommutes[keep_vars]

#change everything to rounded percents
excommutes = excommutes.multiply(100).round(2)
excommutes

In [12]:
#export data to csv
export_path = cwd+'/Output/python_output/commutes.csv'
excommutes = excommutes.to_csv(export_path)